<a href="https://colab.research.google.com/github/katerynanevm/S-P-500-Prediction-model/blob/main/Investing_allocation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from scipy.optimize import minimize

In [31]:
data = {
    "Company": ["Puma", "NVIDIA", "D6H", "Alibaba"],
    "Expected_1y_target": [65.15, 19.94, 80.0, 35.86],
    "Uncertainty": ["Low", "Med", "Low", "Med"],
    "Undervalued": [71.4, -13.5, 46.0, 59.9],
    "Warnings": [0.5714285714, 0.3333333333, 0.2, 0],
    "Financial_health": [2, 4, 3, 3]
}

df = pd.DataFrame(data)

In [32]:
# Categorical variables encoding
df['Uncertainty'] = df['Uncertainty'].map({"Low": 1, "Med": 2, "High": 3})

In [33]:
# Data normalizing for scaling
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(df[["Expected_1y_target", "Undervalued", "Warnings", "Financial_health", "Uncertainty"]])
df_normalized = pd.DataFrame(normalized_data, columns=["Expected_1y_target", "Undervalued", "Warnings", "Financial_health", "Uncertainty"])

In [34]:
# Creation of the function, where we set the formula that we use to decide how to divide money between stocks.
# It tries to maximize good things (like returns and undervaluation) and minimize bad things (like uncertainty and warnings).
# It also adds a penalty if we put too much money into just one stock, to make sure the money is spread out more evenly.

def objective(weights):
    weights = np.array(weights)

    # Importance weights
    expected_target_weight = 5
    undervalued_weight = 4
    warnings_weight = 3
    financial_health_weight = 3
    uncertainty_weight = 2

    # Calculate contributions
    returns = weights.dot(df_normalized["Expected_1y_target"]) * expected_target_weight
    undervaluation = weights.dot(df_normalized["Undervalued"]) * undervalued_weight
    warnings = weights.dot(df_normalized["Warnings"]) * warnings_weight
    financial_health = weights.dot(df_normalized["Financial_health"]) * financial_health_weight
    uncertainty = weights.dot(df_normalized["Uncertainty"]) * uncertainty_weight

    # Diversification penalty: Discourage concentration in a single stock
    diversification_penalty = np.sum(weights**2)  # Squaring makes large weights more penalized

    # Objective: Maximize returns and undervaluation, minimize warnings, uncertainty, and concentration
    return -(returns + undervaluation - warnings - uncertainty + financial_health) + 0.1 * diversification_penalty


In [37]:
# Constraints: Weights sum to 1
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

# Add additional bounds for diversification. # At least 15%, at most 50% for each stock
bounds = [(0.15, 0.5) for _ in range(len(df))]

# Initial weights
initial_weights = [1/len(df)] * len(df)

# Optimize
result = minimize(objective, initial_weights, bounds=bounds, constraints=constraints)

# Get the optimal weights
optimal_weights = result.x
df['Investment'] = optimal_weights * 1000  # Allocate $1000 based on weights

In [38]:
print("You should invest in each stock as follows:")
for company, investment in zip(df["Company"], df["Investment"]):
    print(f"{company}: ${investment:.2f}")

You should invest in each stock as follows:
Puma: $200.00
NVIDIA: $150.00
D6H: $500.00
Alibaba: $150.00
